In [1]:
# boilerplate, initialization
import os, sys, re
import tqdm
import csv
import pandas as pd

args = {
        'input': '/home/gjungwirth/data/02_data/03_analyses/gitleaks_02_own_regexes/analyses/all_emails_compressed.csv',
        'output': '/home/gjungwirth/data/02_data/03_analyses/gitleaks_02_own_regexes/analyses/emails_tfidf.csv'
    }

data = pd.read_csv(args['input'], header=None,
    names=['count_per_file', 'email', 'file_path', 'repo_folder'],
    #keep_default_na=False
    ).dropna()

In [ ]:
data.head()

In [3]:
# testdata = data.head(n=2000)
data.drop(columns=['count_per_file', 'file_path'], inplace=True)
data.columns = ['term', 'document']


In [4]:

## now the tfidf manual


def manual_tfidf(data):
  term_to_freq = {}
  document_to_freq = {}
  for row in data.itertuples():
    term_to_freq.setdefault(row.term, {}).setdefault(row.document, 0)
    term_to_freq[row.term][row.document] += 1

    document_to_freq.setdefault(row.document, {}).setdefault(row.term, 0)
    document_to_freq[row.document][row.term] += 1

  term_to_freq2 = {}
  for item in term_to_freq.items():
    term_to_freq2[item[0]] = len(item[1].keys())

  # data['tfidf_test'] = data.eval('term_to_freq2[term] * document_to_freq[document][term]')
  data['tfidf_test'] = [
            document_to_freq[document][term] / term_to_freq2[term]
            for term, document in zip(data.term, data.document)
  ]
  

manual_tfidf(data)


In [ ]:
#data.sort_values(by='tfidf_test', ascending=False).head()
res = data.groupby(['term','document']).max()

res[res['tfidf_test'] > 3].sort_values(by='tfidf_test',ascending=False).head(n=10)
#res[res['tfidf_test'] > 3].shape()


In [6]:
# here we finalize (clean) the data for output and write it to a csv
# we choose all lines with a tfidf>3, which includes many emails but filters the most obvious low-rated uninteresting stuff

# res[res['tfidf_test'] > 3].shape
# res[res['tfidf_test'] > 3].head()
for_output = res[res['tfidf_test'] > 3] \
    .reset_index() \
    .sort_values(by='tfidf_test',ascending=False) \
    .rename(
        columns={
            'term': 'email',
            'document': 'repo',
            'tfidf_test': 'tfidf'
        }
    )
for_output['tfidf'] = for_output['tfidf'].apply(int)
#for_output.head()
for_output.to_csv(args['output'],index=False)

## References

* https://pandas.pydata.org/docs/getting_started/intro_tutorials/02_read_write.html - the best pandas introduction
* http://www.ultravioletanalytics.com/blog/tf-idf-basics-with-pandas-scikit-learn - pandas and sklearn
* https://code.visualstudio.com/docs/python/data-science-tutorial - everything with VSCode
* https://sondosatwi.wordpress.com/2017/08/01/using-text-data-and-dataframemapper-in-python/ - DataFrameMapper for data preparation
* 

OLD: this was an old attempt to use sklearn

next steps:

1. add a tf field and calculate it
1. make another aggregation and calculate tf.idf?

In [7]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# v = TfidfVectorizer()
# x = v.fit_transform(data['email'])